# Overhead Signs Work Order Creation
--------------------------------------------------
The purpose of this script is to create work orders of overhead street name signs. The information for work orders are created from a feature layer of operational maintenance areas intersecting signalized intersections.

Disclaimer: 

## Imports

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer,GeoAccessor, GeoSeriesAccessor
from arcgis.features.manage_data import overlay_layers
import datetime
import pandas as pd
import openpyxl
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from pathlib import Path
import math

## Constants
Set constants of the notebook.

In [3]:
URL = r"https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/{}/FeatureServer/0"
AUTHOR = "Susanne Gov"
FILE_PATH = str(Path.cwd())
YEAR = 2020
TABLE_NAME = FILE_PATH + r"/OverheadSigns_FY{}.xlsx".format(str(YEAR))

## Part 1: Setup Table

<b>This step can be skipped if the Excel file is already in the folder. </b>

Set variables for analysis layer title and columns for table.

In [4]:
overhead = "Signs Markings Maintenance"
directions = list("NSEW")
col = ['COA_INTERSECTION_ID','FY_OVERHEAD_SIGN_MAINT','LOCATION_NAME','LATITUDE','LONGITUDE']

Acccess COA account

In [ ]:
gis = GIS("https://austin.maps.arcgis.com/home/index.html", client_id='CrnxPfTcm7Y7ZGl7')

Retrieve locations of signalized intersections intersecting operational maintenance areas to publish into a folder and excel worksheet.

In [ ]:
# This intersect analysis has already been completed
#signals = FeatureLayer(URL.format("TRANSPORTATION_signals2"))
#oma = FeatureLayer(URL.format("Signs_Markings_OMA")) #OMA layer will be on AGOL
#analysis = overlay_layers(oma,signals,overlay_type="Intersect", output_name=overhead)

Retrieve overhead feature layer.

In [4]:
fl = pd.DataFrame.spatial.from_layer(FeatureLayer(URL.format(overhead))).set_index("SIGNAL_ID")

Query and filter dataset. Add columns for Google URL, N,S,E,W, and Created By.

In [33]:
df = fl.query("FY_OVERHEAD_SIGN_MAINT == {}".format(str(YEAR)).filter(items=col)
g_url = "https://www.google.com/maps/place/{},+Austin,+TX"
gmap = df['LOCATION_NAME'].apply(lambda x: g_url.format(x.lstrip().replace(" / ","+%26+").replace(" ","+").split("(", 1)[0]))
gmap = pd.DataFrame(gmap).rename(columns={"LOCATION_NAME":"GOOGLE_URL"})
df = pd.concat([df.join(gmap),pd.DataFrame(columns=directions)],sort=False)
df['CREATED_BY'] = AUTHOR
df.to_excel(TABLE_NAME, sheet_name='OMA_Signals')
display(df.head()) # Display first 5 rows

,COA_INTERSECTION_ID,FY_OVERHEAD_SIGN_MAINT,LOCATION_NAME,LATITUDE,LONGITUDE,GOOGLE_URL,N,S,E,W,CREATED_BY
88,5157283.0,2020,BURNET RD / HANCOCK DR,None,None,https://www.google.com/maps/place/BURNET+RD+%2...,NaN,NaN,NaN,NaN,Susanne Gov
104,5155653.0,2020,LAMAR BLVD / PARKWAY,None,None,https://www.google.com/maps/place/LAMAR+BLVD+%...,NaN,NaN,NaN,NaN,Susanne Gov
111,5155479.0,2020,35TH ST / JEFFERSON ST,None,None,https://www.google.com/maps/place/35TH+ST+%26+...,NaN,NaN,NaN,NaN,Susanne Gov
205,5153995.0,2020,6TH ST / CAMPBELL ST,None,None,https://www.google.com/maps/place/6TH+ST+%26+C...,NaN,NaN,NaN,NaN,Susanne Gov
209,5154661.0,2020,12TH ST / WEST LYNN ST,None,None,https://www.google.com/maps/place/12TH+ST+%26+...,NaN,NaN,NaN,NaN,Susanne Gov


In [29]:
# Estimate photos to to input manually. Multiplied by 4 by number of signal poles.
df['GOOGLE_URL'].count() * 4

548

## Part 2: Retrieve Photos from URL

After the excel spreadsheet has been created, <b>manually retrieve the google streetview urls of overhead signal poles for 2020 maintenance.</b> The goal of collecthing these URLS is to eventually generate photos of overhead traffic signs in need of maintenance.

In [30]:
#pretty up later
h = "{visibility: hidden}"
jsfunct = ('var style = document.createElement("style");style.type = "text/css";' +
'style.innerHTML = ".widget-image-header-close {visibility: hidden} .widget-image-header-scrim {visibility: hidden}' +
'.watermark { visibility: hidden;} .app-viewcard-strip {visibility: hidden}' + 
'.scene-footer {visibility: hidden} #titlecard {visibility: hidden}";'+ 
'document.getElementsByTagName("head")[0].appendChild( style );')

df = pd.read_excel(TABLE_NAME,index_col=0)
driver = webdriver.Chrome(FILE_PATH + r'/chromedriver')
driver.maximize_window() 

# for loop 548 links. expection if null link
i=0
for index, row in df.iterrows():
    for x in directions:
        link = row[x]
        if type(link) == str and link != "N/A":
            driver.get(link)
            driver.execute_script(jsfunct);
            # To do: Set screenshot sizing/croping for work order. Test for save photo file path.
            # If successful, can remove PIL and resizeimage package
            time.sleep(5) # wait for page to load
            screenshot = driver.save_screenshot(FILE_PATH + r'/SignPhotos/{}_{}.png'.format(index,x))
    if i == 5:
        break #this is for testing. remove line after test is complete.
    i += 1
driver.quit()

In [30]:
display(df)

,Unnamed: 0,COA_INTERSECTION_ID,FY_OVERHEAD_SIGN_MAINT,LOCATION_NAME,LATITUDE,LONGITUDE,GOOGLE_URL,LINK,N,S,E,W,CREATED_BY
0,88,5157283,2020,BURNET RD / HANCOCK DR,NaN,NaN,https://www.google.com/maps/place/BURNET+RD+%2...,https://www.google.com/maps/place/BURNET+RD+%2...,NaN,NaN,NaN,NaN,Susanne Gov
1,104,5155653,2020,LAMAR BLVD / PARKWAY,NaN,NaN,https://www.google.com/maps/place/LAMAR+BLVD+%...,https://www.google.com/maps/place/LAMAR+BLVD+%...,https://goo.gl/maps/52cUefr9qhs1Jb2U8,https://goo.gl/maps/kh62J1KTq46C6cpT6,NaN,NaN,Susanne Gov
2,111,5155479,2020,35TH ST / JEFFERSON ST,NaN,NaN,https://www.google.com/maps/place/35TH+ST+%26+...,https://www.google.com/maps/place/35TH+ST+%26+...,https://goo.gl/maps/DpaB8rqj2Lf2U2av9,https://goo.gl/maps/vJsALCV3zJKce9cBA,https://goo.gl/maps/SQMjudY1dz4NRNnv9,https://goo.gl/maps/YAUA7TnakpgK61Hy7,Susanne Gov
3,205,5153995,2020,6TH ST / CAMPBELL ST,NaN,NaN,https://www.google.com/maps/place/6TH+ST+%26+C...,https://www.google.com/maps/place/6TH+ST+%26+C...,https://goo.gl/maps/KXQWi8eaqSrZU4ug8,https://goo.gl/maps/Pyx1rpRorPyN512w5,NaN,https://goo.gl/maps/mdM4HZ2VK6WXGqiUA,Susanne Gov
4,209,5154661,2020,12TH ST / WEST LYNN ST,NaN,NaN,https://www.google.com/maps/place/12TH+ST+%26+...,https://www.google.com/maps/place/12TH+ST+%26+...,NaN,NaN,NaN,NaN,Susanne Gov
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,206,5146140,2020,SOUTHWEST PKWY / FOSTER RANCH RD,NaN,NaN,https://www.google.com/maps/place/SOUTHWEST+PK...,https://www.google.com/maps/place/SOUTHWEST+PK...,NaN,NaN,NaN,NaN,Susanne Gov
133,125,5152101,2020,LAMAR BLVD / MENCHACA RD (Barton Skwy),NaN,NaN,https://www.google.com/maps/place/LAMAR+BLVD+%...,https://www.google.com/maps/place/LAMAR+BLVD+%...,NaN,NaN,NaN,NaN,Susanne Gov
134,127,5151991,2020,LAMAR BLVD / BARTON SKWY (Menchaca Rd),NaN,NaN,https://www.google.com/maps/place/LAMAR+BLVD+%...,https://www.google.com/maps/place/LAMAR+BLVD+%...,NaN,NaN,NaN,NaN,Susanne Gov
135,530,5147905,2020,SOUTHWEST PKWY / REPUBLIC OF TEXAS BLVD,NaN,NaN,https://www.google.com/maps/place/SOUTHWEST+PK...,https://www.google.com/maps/place/SOUTHWEST+PK...,NaN,NaN,NaN,NaN,Susanne Gov


In [18]:
# Workflow for 2020
# 1) Merge OMA and signals - waiting for OMA to be published on AGOL/GISMaint
# 2) Filter df to 2020 Overhead maintenance
# 3) Create dataset of all urls
# 4) Input overhead photo urls
# 5) Create for loop for screengrab, cropping, and input into Vision AI api
# 6) Get JSON for each photo
# 6) Create table of work orders from JSON
#
#
# VVVV              BELOW IS PART OF OLD SCRIPT             VVVV

In [18]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from PIL import Image
from resizeimage import resizeimage
from collections import OrderedDict
import datetime
import sys
import os
import win32com.client

In [5]:
# This will create the list of values for each intersection based on signal ID
filePath = r"G:\ATD\ATD_GIS\Signs\123_Signs_Maintenance_Plan\Operational_Maintenance_Areas_Sign_Maintenance_Plan"
author = "Susanne Gov"

df = pd.read_excel(filePath + r"\OverheadSigns_FY2019.xlsx", sheet_name='Signs')  # sheetname is optional
df.to_csv(filePath + r'\OverheadSigns_FY2019_SIGNS.csv', index=False)

work_orders = pd.read_excel(filePath + r"\OverheadSigns_FY2019.xlsx", sheet_name='StreetView Link')  # sheetname is optional
work_orders.to_csv(filePath + r'\OverheadSigns_FY2019.csv', index=False)          
signalID = []
index = 1
with open (filePath + r'\OverheadSigns_FY2019.csv','r') as csvfile:
    orderRow = csv.reader(csvfile, delimiter=',')
    for row in orderRow:
        values = OrderedDict()
        if row[0] != "SIGNAL_ID":
            values["Created Date"] = row[15]
            values["Created By"] = author
            values["Work Order ID"] = row[14]
            values["Signal ID"] = row[0]
            values["Intersection ID"] = row[1]
            values["Primary Street"] = row[2]
            values["Cross Street"] = row[3]
            values["Link (N)"] = row[10]
            values["Link (S)"] = row[11]
            values["Link (E)"] = row[12]
            values["Link (W)"] = row[13]
            signalID.append(values)
            index +=1
    index = 1
    csvfile.close()

In [3]:
signs = []

with open (filePath + r'\OverheadSigns_FY2019_SIGNS.csv','rU') as csvfile:
    csv = csv.reader(csvfile, delimiter=',')
    for row in csv:
        sign = OrderedDict()
        sign["Signal ID"] = row[0]
        sign["Direction"] = row[1]
        sign["Sign Type"] = row[2] 
        sign["Street Sign"] = row[3]
        sign["Bottom Text (Optional)"] = row[6]
        sign["Install/Remove"] = row[7]
        signs.append(sign)

In [4]:
# Take an existing image of the streetview imagery and will resize the image to fit the work order template
def makeImg (sigId, nSlide):
    imgname = sigId + "_" + direction
    imagery = imagePath + "/" + imgname + ".png"
    try:
        with open(imagery, 'r+b') as f:
            with Image.open(f) as image:
                cover = resizeimage.resize_contain(image, [560, 280])
                cover.save(imagePath + "Cropped/" + imgname + ".png", image.format)
        pic = nSlide.add_picture(imagePath + "Cropped/" + imgname + ".png",Inches(0.25),Inches(1.1))
        line = pic.line
        line.color.rgb = RGBColor(0,0,0)
        line.width = Inches(0.05)
    except IOError:
        return None        

In [5]:
# Create a pretty header
def makeHead(nSlide):
    title = nSlide.add_textbox(Inches(0.25),Inches(0.4),Inches(7.5),Inches(0.75))
    tf = title.text_frame
    tf.clear()
    p = tf.paragraphs[0]
    run = p.add_run()
    run.text = "Austin Transportation Department Signs Work Orders"
    font = run.font
    font.name = 'Arial'
    font.size = Pt(20)
    font.bold = True
    seal = "G:/ATD/ATD_GIS/03_RESOURCES/Seals_Logos/cityseal_60_x_60.jpg"
    pic = nSlide.add_picture(seal,Inches(8.5),Inches(0.25))

In [6]:
# Create information box for work orders
def makeInfo(sigID, nSlide, direction):
    info = nSlide.add_textbox(Inches(8.25),Inches(1.15),Inches(1.5),Inches(4.5))
    infotext = info.text_frame.paragraphs[0]
    info.text_frame.word_wrap = True
    for (key,val) in sigID.iteritems():
        if "Link" not in key or direction in key:
            run1 = infotext.add_run()
            run1.text = key + "\n"
            run1.font.size = Pt(12)
            run1.font.bold = True
            run1.font.underline = True
            run2 = infotext.add_run();
            if "Link" in key and "N/A" not in val:
                run2.hyperlink.address = val
            run2.text =  val + "\n"
            run2.font.size = Pt(12)
    run3 = infotext.add_run()
    run3.text = "Cardinal Direction: "
    run3.font.bold = True
    run3.font.underline = True  
    run3.font.size = Pt(12)
    c = infotext.add_run()
    if direction is "N":
        c.text = "North"
    elif (direction is "S"):
        c.text = "South"
    elif (direction is "E"):
        c.text = "East"
    elif (direction is "W"):
        c.text = "West"
    c.font.size = Pt(12)   

In [7]:
# For changing table font sizes
def iter_cells(table):
    for row in table.rows:
        for cell in row.cells:
            yield cell
# Create list of Sign Overheads
def makeSignTable(sigID,nSlide,direction,signs):
    # row/column/left/top/width/height
    table = nSlide.add_table(6,4,Inches(0.25),Inches(5.2),170,1000).table
    table.cell(0,0).text = "Sign Type"
    table.cell(0,1).text = "Street Sign"
    table.cell(0,2).text = "Bottom Text"
    table.cell(0,3).text = "Install/Remove"
    
    table.columns[0].width = Inches(5)
    table.columns[1].width = Inches(1.75)
    table.columns[2].width = Inches(1.5)
    table.columns[3].width = Inches(1.3)
    
    index = 1
    for x in signs:
        if x["Signal ID"] == sigID and x["Direction"] == direction:
            table.cell(index,0).text = x["Sign Type"]
            table.cell(index,1).text = x["Street Sign"]
            table.cell(index,2).text = x["Bottom Text (Optional)"]
            table.cell(index,3).text = x["Install/Remove"]
            index +=1
    for cell in iter_cells(table):
        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(12)

In [8]:
# Converts Powerpoint file to PDF file
def convertPDF(path,name):
    in_file = path + "/" + name
    out_file = path + "PDF\\" + name[:-5] + ".pdf"
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")
    pdf = powerpoint.Presentations.Open(in_file,WithWindow=False)
    pdf.SaveCopyAs(out_file,32)
    pdf.Close()
    powerpoint.Quit()

In [9]:
dirs = ["N","S","E","W"]
path = "G:\\ATD\\ATD_GIS\\Signs\\123_Signs_Maintenance_Plan\\Operational_Maintenance_Areas_Sign_Maintenance_Plan\\WorkOrderSignsOverheadFY2019"
imagePath = "G:/ATD/ATD_GIS/Signs/123_Signs_Maintenance_Plan/Operational_Maintenance_Areas_Sign_Maintenance_Plan/SignsFY2019Imagery"

for valueDictionary in signalID:
    workOrder = Presentation()
    dirindex = 0
    for direction in dirs:
        if "N/A" not in valueDictionary["Link ("+ direction + ")"]:
            blank_slide_layout = workOrder.slide_layouts[6]
            slide = workOrder.slides.add_slide(blank_slide_layout)
            nSlide = slide.shapes
            makeImg(valueDictionary["Signal ID"], nSlide)
            makeSignTable(valueDictionary["Signal ID"],nSlide,direction,signs)
            makeHead(nSlide)
            makeInfo(valueDictionary, nSlide, direction)
        dirindex += 1
    name = "ATDSignsWorkOrders_" + valueDictionary["Signal ID"] + ".pptx"
    workOrder.save(str(path) + "/" + name)
    convertPDF(path,name)